# Distilling Knowledge in Multiple Students Using Generative Models

In [1]:
# %tensorflow_version 1.x
# !pip install --upgrade opencv-python==3.4.2.17
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
# import os
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
# import keras.backend as K
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils
from tensorflow.keras import utils as np_utils



In [2]:
#Loading and splitting the dataset into train, validation and test
nb_classes = 10

(X_Train, y_Train), (X_test, y_test) = fashion_mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
# convert y_train and y_test to categorical binary values 
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

X_train = X_train.reshape(48000, 28, 28, 1)
X_val = X_val.reshape(12000, 28, 28, 1)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255


#Creating a teacher network
input_shape = (28, 28, 1) # Input shape of each image

teacher = Sequential()
teacher.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
teacher.add(Conv2D(64, (3, 3), activation='relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))

teacher.add(Dropout(0.25)) # For reguralization

teacher.add(Flatten())
teacher.add(Dense(256, activation='relu'))
teacher.add(Dense(256, activation='relu', name="dense_1"))

teacher.add(Dropout(0.5)) # For reguralization

teacher.add(Dense(nb_classes, name = 'dense_2'))
teacher.add(Activation('softmax')) # Note that we add a normal softmax layer to begin with

teacher.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(teacher.summary())

# Train the teacher model as usual
# epochs = 30
# batch_size = 256
# teacher.fit(X_train, Y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(X_val, Y_val))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2359552   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6

In [3]:
teacher.load_weights("Teacher_FMNIST_92.h5")

In [4]:
Y_test = np_utils.to_categorical(y_test, nb_classes)
X_test = X_test.reshape(10000, 28, 28, 1)
X_test = X_test.astype('float32')
# Normalize the values
X_test /= 255
teacher.evaluate(X_test,Y_test)

313/313 [==============================] - 3s 4ms/step - loss: 0.3734 - accuracy: 0.9195


[0.3516419529914856, 0.9239000082015991]

In [6]:
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)


# 8 Students case
# ---------------------------------------------
s1Train=train_dense[:,:32]
s2Train=train_dense[:,32:64]
s3Train=train_dense[:,64:96]
s4Train=train_dense[:,96:128]
s5Train=train_dense[:,128:160]
s6Train=train_dense[:,160:192]
s7Train=train_dense[:,192:224]
s8Train=train_dense[:,224:]

s1Val=val_dense[:,:32]
s2Val=val_dense[:,32:64]
s3Val=val_dense[:,64:96]
s4Val=val_dense[:,96:128]
s5Val=val_dense[:,128:160]
s6Val=val_dense[:,160:192]
s7Val=val_dense[:,192:224]
s8Val=val_dense[:,224:]


In [7]:
def define_model(name):

    student1 = Sequential()
    student1.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 1),kernel_initializer='normal', name=name))
    student1.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',kernel_initializer='normal'))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Dropout(0.25)) # For reguralization
    student1.add(Flatten())
    student1.add(Dense(16, activation='relu'))
    student1.add(Dropout(0.3))
    student1.add(Dense(32,name='req'+name))

    student1.compile('adam',loss='mean_squared_error',metrics=['accuracy'])

    return student1



In [8]:
s1=define_model("s1")
s2=define_model('s2')
s3=define_model("s3")
s4=define_model('s4')
s5=define_model("s5")
s6=define_model('s6')
s7=define_model("s7")
s8=define_model('s8')

In [9]:
batch_size = 128
s1.fit([X_train], s1Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val], s1Val))

s2.fit([X_train], s2Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val], s2Val))

s3.fit([X_train], s3Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val], s3Val))

s4.fit([X_train], s4Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val], s4Val))

s5.fit([X_train], s5Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val], s5Val))

s6.fit([X_train], s6Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val], s6Val))

s7.fit([X_train], s7Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val], s7Val))

s8.fit([X_train], s8Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val], s8Val))

Epoch 1/10
375/375 [==============================] - 4s 10ms/step - loss: 0.9848 - accuracy: 0.1551 - val_loss: 0.3917 - val_accuracy: 0.4513
Epoch 2/10
375/375 [==============================] - 3s 8ms/step - loss: 0.4703 - accuracy: 0.3917 - val_loss: 0.2868 - val_accuracy: 0.4922
Epoch 3/10
375/375 [==============================] - 3s 7ms/step - loss: 0.3974 - accuracy: 0.4311 - val_loss: 0.2702 - val_accuracy: 0.5174
Epoch 4/10
375/375 [==============================] - 3s 8ms/step - loss: 0.3725 - accuracy: 0.4491 - val_loss: 0.2293 - val_accuracy: 0.5328
Epoch 5/10
375/375 [==============================] - 2s 5ms/step - loss: 0.3630 - accuracy: 0.4634 - val_loss: 0.2267 - val_accuracy: 0.5508
Epoch 6/10
375/375 [==============================] - 3s 8ms/step - loss: 0.3546 - accuracy: 0.4736 - val_loss: 0.2296 - val_accuracy: 0.5573
Epoch 7/10
375/375 [==============================] - 3s 9ms/step - loss: 0.3463 - accuracy: 0.4772 - val_loss: 0.2288 - val_accuracy: 0.5592
Epoch

375/375 [==============================] - 4s 11ms/step - loss: 0.4086 - accuracy: 0.5289 - val_loss: 0.2659 - val_accuracy: 0.6168
Epoch 9/10
375/375 [==============================] - 4s 10ms/step - loss: 0.4015 - accuracy: 0.5283 - val_loss: 0.2693 - val_accuracy: 0.6237
Epoch 10/10
375/375 [==============================] - 4s 10ms/step - loss: 0.3958 - accuracy: 0.5339 - val_loss: 0.2533 - val_accuracy: 0.6208
Epoch 1/10
375/375 [==============================] - 4s 9ms/step - loss: 1.1783 - accuracy: 0.0729 - val_loss: 0.4057 - val_accuracy: 0.3008
Epoch 2/10
375/375 [==============================] - 4s 10ms/step - loss: 0.5352 - accuracy: 0.2567 - val_loss: 0.3298 - val_accuracy: 0.3991
Epoch 3/10
375/375 [==============================] - 4s 10ms/step - loss: 0.4630 - accuracy: 0.3252 - val_loss: 0.3169 - val_accuracy: 0.4369
Epoch 4/10
375/375 [==============================] - 4s 10ms/step - loss: 0.4302 - accuracy: 0.3486 - val_loss: 0.2866 - val_accuracy: 0.4456
Epoch 5/10

### **8 Students**

In [10]:
o1=s1.get_layer("reqs1").output
o2=s2.get_layer("reqs2").output
o3=s3.get_layer("reqs3").output
o4=s4.get_layer("reqs4").output

o5=s5.get_layer("reqs5").output
o6=s6.get_layer("reqs6").output
o7=s7.get_layer("reqs7").output
o8=s8.get_layer("reqs8").output

output=tensorflow.keras.layers.concatenate([o1,o2,o3,o4, o5,o6,o7,o8])
print (output.shape)
output=Activation('relu')(output)
output2=Dropout(0.3)(output) # For reguralization
output3=Dense(10, activation="softmax", name="d1")(output2)
multi_model=Model([s1.get_layer("s1").input,s2.get_layer("s2").input,s3.get_layer("s3").input,s4.get_layer("s4").input,
                  s5.get_layer("s5").input, s6.get_layer("s6").input,s7.get_layer("s7").input,s8.get_layer("s8").input],output3)
my_weights=teacher.get_layer('dense_2').get_weights()
multi_model.get_layer("d1").set_weights(my_weights)

(None, 256)


In [11]:
for l in multi_model.layers[:len(multi_model.layers)-1]:
    l.trainable=False

In [12]:
multi_model.compile(loss='categorical_crossentropy',
            optimizer='nadam',
            metrics=['accuracy'])

In [13]:
batch_size = 128
mm_history=multi_model.fit([X_train,X_train,X_train,X_train,X_train,X_train,X_train,X_train], Y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val, X_val, X_val, X_val,X_val, X_val, X_val, X_val], Y_val))

Epoch 1/10
375/375 [==============================] - 6s 12ms/step - loss: 0.4945 - accuracy: 0.8319 - val_loss: 0.3650 - val_accuracy: 0.8666
Epoch 2/10
375/375 [==============================] - 5s 12ms/step - loss: 0.4394 - accuracy: 0.8390 - val_loss: 0.3481 - val_accuracy: 0.8685
Epoch 3/10
375/375 [==============================] - 4s 11ms/step - loss: 0.4244 - accuracy: 0.8390 - val_loss: 0.3414 - val_accuracy: 0.8707
Epoch 4/10
375/375 [==============================] - 5s 12ms/step - loss: 0.4202 - accuracy: 0.8418 - val_loss: 0.3403 - val_accuracy: 0.8707
Epoch 5/10
375/375 [==============================] - 5s 12ms/step - loss: 0.4112 - accuracy: 0.8468 - val_loss: 0.3408 - val_accuracy: 0.8712
Epoch 6/10
375/375 [==============================] - 4s 11ms/step - loss: 0.4037 - accuracy: 0.8482 - val_loss: 0.3373 - val_accuracy: 0.8727
Epoch 7/10
375/375 [==============================] - 4s 12ms/step - loss: 0.4098 - accuracy: 0.8423 - val_loss: 0.3377 - val_accuracy: 0.8730

In [14]:
loss, acc = multi_model.evaluate([X_test,X_test,X_test,X_test,X_test,X_test,X_test,X_test],Y_test)
loss, acc

313/313 [==============================] - 6s 18ms/step - loss: 0.3670 - accuracy: 0.8615


(0.3670448362827301, 0.8615000247955322)